## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import warnings

## Basic Important Settings

In [2]:
pio.renderers.default = 'iframe'
warnings.filterwarnings("ignore")

## Loading the Dataset

In [3]:
df = pd.read_csv("/kaggle/input/intel-stock-data-1980-2024/data.csv")

## Exploring the Dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11289 entries, 0 to 11288
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          11289 non-null  object 
 1   Open          11289 non-null  float64
 2   High          11289 non-null  float64
 3   Low           11289 non-null  float64
 4   Close         11289 non-null  float64
 5   Volume        11289 non-null  int64  
 6   Dividends     11289 non-null  float64
 7   Stock Splits  11289 non-null  float64
dtypes: float64(6), int64(1), object(1)
memory usage: 705.7+ KB


In [5]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,1980-03-17 00:00:00-05:00,0.181500,0.184404,0.181500,0.181500,10924800,0.0,0.0
1,1980-03-18 00:00:00-05:00,0.181500,0.182952,0.180048,0.180048,17068800,0.0,0.0
2,1980-03-19 00:00:00-05:00,0.184404,0.187308,0.184404,0.184404,18508800,0.0,0.0
3,1980-03-20 00:00:00-05:00,0.184403,0.186581,0.183678,0.183678,11174400,0.0,0.0
4,1980-03-21 00:00:00-05:00,0.180048,0.180048,0.177143,0.177143,12172800,0.0,0.0


## Data Preprocessing

In [6]:
df['Date'] = pd.to_datetime(df['Date'], utc=True)

df['Hour'] = df['Date'].dt.hour
df['Minute'] = df['Date'].dt.minute
df['Second'] = df['Date'].dt.second

df['UTC_Offset'] = df['Date'].dt.tz

# Separate day, month, and year
df['Day'] = df['Date'].dt.day
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year


In [7]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Hour,Minute,Second,UTC_Offset,Day,Month,Year
0,1980-03-17 05:00:00+00:00,0.181500,0.184404,0.181500,0.181500,10924800,0.0,0.0,5,0,0,UTC,17,3,1980
1,1980-03-18 05:00:00+00:00,0.181500,0.182952,0.180048,0.180048,17068800,0.0,0.0,5,0,0,UTC,18,3,1980
2,1980-03-19 05:00:00+00:00,0.184404,0.187308,0.184404,0.184404,18508800,0.0,0.0,5,0,0,UTC,19,3,1980
3,1980-03-20 05:00:00+00:00,0.184403,0.186581,0.183678,0.183678,11174400,0.0,0.0,5,0,0,UTC,20,3,1980
4,1980-03-21 05:00:00+00:00,0.180048,0.180048,0.177143,0.177143,12172800,0.0,0.0,5,0,0,UTC,21,3,1980


## Exploratory Data Analysis (EDA)

### General Data Overview

**Missing Values**

In [8]:
df.isnull().sum()

Date            0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
Hour            0
Minute          0
Second          0
UTC_Offset      0
Day             0
Month           0
Year            0
dtype: int64

> **No Missing Values**

**Duplicates**

In [9]:
df.duplicated().sum()

0

> **No Duplicates**

**Outliers**

In [10]:
fig_boxplot = px.box(df, y=['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits'],
                     title="Boxplots for Numerical Columns",
                     template="plotly_dark")
fig_boxplot.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_boxplot.show()

**Data Range (Earliest to Latest Date)**

In [11]:
date_range = [df['Date'].min(), df['Date'].max()]

fig_date_range = go.Figure(
    data=[go.Scatter(x=date_range, y=[0, 0], mode='lines+markers', marker=dict(color='green'))],
    layout=go.Layout(
        title="Date Range (Earliest to Latest)",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Value"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_date_range.show()

**Coverage Consistency - Checking for any gaps in dates**

In [12]:
date_coverage = pd.date_range(start=df['Date'].min(), end=df['Date'].max(), freq='D')
missing_dates = set(date_coverage) - set(df['Date'])

fig_coverage = go.Figure(
    data=[go.Scatter(x=list(missing_dates), y=[1]*len(missing_dates), mode='markers', marker=dict(color='orange'))],
    layout=go.Layout(
        title="Missing Date Coverage",
        xaxis=dict(title="Missing Dates"),
        yaxis=dict(title="Missing Indicator"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_coverage.show()

### Trend Analysis

**Overall Trend of Close Prices**

In [13]:
fig_trend = go.Figure(
    data=[go.Scatter(x=df['Date'], y=df['Close'], mode='lines', line=dict(color='cyan'))],
    layout=go.Layout(
        title="Overall Trend of Close Prices",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Close Price"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_trend.show()

**Periods of Significant Growth or Decline**

In [14]:
fig_growth_decline = go.Figure()

# Plotting the general trend line
fig_growth_decline.add_trace(
    go.Scatter(x=df['Date'], y=df['Close'], mode='lines', line=dict(color='cyan'), name="Close Price")
)

# Highlight the period of significant growth (2019-2020)
fig_growth_decline.add_trace(
    go.Scatter(x=df[df['Date'].dt.year.between(2019, 2020)]['Date'],
               y=df[df['Date'].dt.year.between(2019, 2020)]['Close'],
               mode='lines',
               line=dict(color='lime', width=3),
               name="Growth 2019-2020")
)

# Highlight the period of significant decline (2008 Financial Crisis)
fig_growth_decline.add_trace(
    go.Scatter(x=df[df['Date'].dt.year == 2008]['Date'],
               y=df[df['Date'].dt.year == 2008]['Close'],
               mode='lines',
               line=dict(color='red', width=3),
               name="Decline 2008")
)

fig_growth_decline.update_layout(
    title="Periods of Significant Growth or Decline",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Close Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_growth_decline.show()

**Trends Before and After Major Financial Crises**

In [15]:
fig_crisis_comparison = go.Figure()

# Plotting the general trend line
fig_crisis_comparison.add_trace(
    go.Scatter(x=df['Date'], y=df['Close'], mode='lines', line=dict(color='cyan'), name="Close Price")
)

# Highlight the period before the 2008 crisis (2007)
fig_crisis_comparison.add_trace(
    go.Scatter(x=df[df['Date'].dt.year == 2007]['Date'],
               y=df[df['Date'].dt.year == 2007]['Close'],
               mode='lines',
               line=dict(color='orange', width=3),
               name="Before 2008 Crisis")
)

# Highlight the period after the 2008 crisis (2009 onwards)
fig_crisis_comparison.add_trace(
    go.Scatter(x=df[df['Date'].dt.year >= 2009]['Date'],
               y=df[df['Date'].dt.year >= 2009]['Close'],
               mode='lines',
               line=dict(color='yellow', width=3),
               name="After 2008 Crisis")
)

# Highlight the period before the COVID-19 market dip (2019)
fig_crisis_comparison.add_trace(
    go.Scatter(x=df[df['Date'].dt.year == 2019]['Date'],
               y=df[df['Date'].dt.year == 2019]['Close'],
               mode='lines',
               line=dict(color='purple', width=3),
               name="Before COVID-19")
)

# Highlight the period after the COVID-19 market dip (2020 onwards)
fig_crisis_comparison.add_trace(
    go.Scatter(x=df[df['Date'].dt.year >= 2020]['Date'],
               y=df[df['Date'].dt.year >= 2020]['Close'],
               mode='lines',
               line=dict(color='pink', width=3),
               name="After COVID-19")
)

# Layout update
fig_crisis_comparison.update_layout(
    title="Trends Before and After Major Financial Crises",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Close Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_crisis_comparison.show()

### Volatility Analysis

In [16]:
# Calculate Daily Price Range (Volatility)
df['Volatility'] = df['High'] - df['Low']

**Plotting the Daily Price Range (Volatility)**

In [17]:
fig_volatility = go.Figure(
    data=[go.Scatter(x=df['Date'], y=df['Volatility'], mode='lines', line=dict(color='cyan'))],
    layout=go.Layout(
        title="Daily Price Range (Volatility)",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Price Range (High - Low)"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_volatility.show()

**Identify Periods of High Volatility**

In [18]:
high_volatility_threshold = df['Volatility'].quantile(0.90)
high_volatility_days = df[df['Volatility'] > high_volatility_threshold]

In [19]:
# Plotting the general volatility and high volatility periods
fig_high_volatility = go.Figure()

# Plotting the general volatility line
fig_high_volatility.add_trace(
    go.Scatter(x=df['Date'], y=df['Volatility'], mode='lines', line=dict(color='cyan'), name="Volatility")
)

# Highlighting high volatility periods
fig_high_volatility.add_trace(
    go.Scatter(x=high_volatility_days['Date'], y=high_volatility_days['Volatility'],
               mode='markers', marker=dict(color='red', size=8), name="High Volatility")
)

fig_high_volatility.update_layout(
    title="Periods of High Volatility",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price Range (High - Low)"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_high_volatility.show()

**Plotting Yearly Average Volatility**

In [20]:
monthly_volatility = df.groupby('Month')['Volatility'].mean()
yearly_volatility = df.groupby('Year')['Volatility'].mean()

In [21]:
fig_yearly_volatility = go.Figure(
    data=[go.Bar(x=yearly_volatility.index, y=yearly_volatility, marker=dict(color='cyan'))],
    layout=go.Layout(
        title="Yearly Average Volatility",
        xaxis=dict(title="Year"),
        yaxis=dict(title="Average Volatility (High - Low)"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_yearly_volatility.show()

**Plotting Monthly Average Volatility**

In [22]:
fig_monthly_volatility = go.Figure(
    data=[go.Bar(x=monthly_volatility.index.astype(str), y=monthly_volatility, marker=dict(color='magenta'))],
    layout=go.Layout(
        title="Monthly Average Volatility",
        xaxis=dict(title="Month"),
        yaxis=dict(title="Average Volatility (High - Low)"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_monthly_volatility.show()

### Volume Analysis

**Plotting the Trend of Trading Volume Over Time**

In [23]:
fig_volume_trend = go.Figure(
    data=[go.Scatter(x=df['Date'], y=df['Volume'], mode='lines', line=dict(color='cyan'))],
    layout=go.Layout(
        title="Trend of Trading Volume Over Time",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Trading Volume"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_volume_trend.show()

**Identify Days with Unusually High or Low Trading Activity**

In [24]:
# Similarly, unusually low trading volume will be defined as days where the volume is below the 10th percentile.
high_volume_threshold = df['Volume'].quantile(0.90)
low_volume_threshold = df['Volume'].quantile(0.10)

# Extracting high and low volume days
high_volume_days = df[df['Volume'] > high_volume_threshold]
low_volume_days = df[df['Volume'] < low_volume_threshold]

**Plotting the Volume Trend with High and Low Volume Days Highlighted**

In [25]:
fig_volume_activity = go.Figure()

# Plotting the general trading volume line
fig_volume_activity.add_trace(
    go.Scatter(x=df['Date'], y=df['Volume'], mode='lines', line=dict(color='cyan'), name="Volume Trend")
)

# Highlighting high volume periods
fig_volume_activity.add_trace(
    go.Scatter(x=high_volume_days['Date'], y=high_volume_days['Volume'],
               mode='markers', marker=dict(color='red', size=8), name="High Volume Days")
)

# Highlighting low volume periods
fig_volume_activity.add_trace(
    go.Scatter(x=low_volume_days['Date'], y=low_volume_days['Volume'],
               mode='markers', marker=dict(color='yellow', size=8), name="Low Volume Days")
)

fig_volume_activity.update_layout(
    title="Periods of High and Low Trading Volume",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Trading Volume"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_volume_activity.show()

### Dividends and Stock Splits

**Plotting Dividend Payouts Over the Years**

In [26]:
dividend_df = df[df['Dividends'] > 0]  # Filter rows where dividends are paid

fig_dividends = go.Figure(
    data=[go.Scatter(x=dividend_df['Date'], y=dividend_df['Dividends'], 
                     mode='markers', marker=dict(color='green', size=10), name="Dividends Paid")],
    layout=go.Layout(
        title="Dividend Payouts Over the Years",
        xaxis=dict(title="Date"),
        yaxis=dict(title="Dividend Amount"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_dividends.show()

In [27]:
# Identify stock splits where the ratio is greater than 1 (indicating a split occurred)
stock_split_df = df[df['Stock Splits'] > 1]

fig_stock_splits = go.Figure()

# Plotting Close Prices
fig_stock_splits.add_trace(
    go.Scatter(x=df['Date'], y=df['Close'], mode='lines', line=dict(color='cyan'), name="Close Price")
)

# Plotting Volume
fig_stock_splits.add_trace(
    go.Scatter(x=df['Date'], y=df['Volume'], mode='lines', line=dict(color='yellow'), name="Volume")
)

# Highlighting Stock Splits
fig_stock_splits.add_trace(
    go.Scatter(x=stock_split_df['Date'], y=stock_split_df['Close'],
               mode='markers', marker=dict(color='red', size=10), name="Stock Splits")
)

fig_stock_splits.update_layout(
    title="Impact of Stock Splits on Stock Prices and Volume",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price and Volume"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_stock_splits.show()

**Significant Stock Splits and Price Trends**

In [28]:
# We will highlight significant splits and show their correlation with stock prices.
significant_splits = stock_split_df[stock_split_df['Stock Splits'] > 2]


fig_significant_splits = go.Figure()

# Plotting Close Price
fig_significant_splits.add_trace(
    go.Scatter(x=df['Date'], y=df['Close'], mode='lines', line=dict(color='cyan'), name="Close Price")
)

# Highlighting Significant Stock Splits
fig_significant_splits.add_trace(
    go.Scatter(x=significant_splits['Date'], y=significant_splits['Close'],
               mode='markers', marker=dict(color='yellow', size=10), name="Significant Stock Splits")
)

fig_significant_splits.update_layout(
    title="Significant Stock Splits and Price Trends",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Stock Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white')
)
fig_significant_splits.show()

### Seasonality and Periodic Analysis

**Plotting Monthly Average Close Prices**

In [29]:
monthly_avg = df.groupby('Month')['Close'].mean()

fig_monthly = go.Figure(
    data=[go.Bar(x=monthly_avg.index, y=monthly_avg.values, marker=dict(color='blue'))],
    layout=go.Layout(
        title="Monthly Average Close Prices",
        xaxis=dict(title="Month"),
        yaxis=dict(title="Average Close Price"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_monthly.show()

**Plotting Monthly average Volume**

In [30]:
monthly_avg_vol = df.groupby('Month')['Volume'].mean()

fig_monthly_vol = go.Figure(
    data=[go.Bar(x=monthly_avg_vol.index, y=monthly_avg_vol.values, marker=dict(color='red'))],
    layout=go.Layout(
        title="Monthly Average Volume",
        xaxis=dict(title="Month"),
        yaxis=dict(title="Average Volume"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_monthly_vol.show()

### Day-wise Average Close Prices 

In [31]:
daily_avg_close = df.groupby('Day')['Close'].mean()

fig_daily_close = go.Figure(
    data=[go.Bar(x=daily_avg_close.index, y=daily_avg_close.values, marker=dict(color='orange'))],
    layout=go.Layout(
        title="Daily Average Close Price",
        xaxis=dict(title="Day"),
        yaxis=dict(title="Average Close Price"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_daily_close.show()

**Day-wise Average Volumes**

In [32]:
daily_avg_volume = df.groupby('Day')['Volume'].mean()

fig_daily_volume = go.Figure(
    data=[go.Bar(x=daily_avg_volume.index, y=daily_avg_volume.values, marker=dict(color='purple'))],
    layout=go.Layout(
        title="Daily Average Volume",
        xaxis=dict(title="Day"),
        yaxis=dict(title="Average Volume"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_daily_volume.show()

**Year-wise Average Close Prices**

In [33]:
yearly_avg_price = df.groupby('Year')['Close'].mean()

fig_yearly_price = go.Figure(
    data=[go.Bar(x=yearly_avg_price.index, y=yearly_avg_price.values, marker=dict(color='lime'))],
    layout=go.Layout(
        title="Year-wise Average Close Prices",
        xaxis=dict(title="Year"),
        yaxis=dict(title="Average Close Price"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_yearly_price.show()

**Year-wise Average Trading Volumes**

In [34]:
yearly_avg_volume = df.groupby('Year')['Volume'].mean()

fig_yearly_volume = go.Figure(
    data=[go.Bar(x=yearly_avg_volume.index, y=yearly_avg_volume.values, marker=dict(color='magenta'))],
    layout=go.Layout(
        title="Year-wise Average Trading Volumes",
        xaxis=dict(title="Year"),
        yaxis=dict(title="Average Trading Volume"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white')
    )
)
fig_yearly_volume.show()

**Correlation heatmap for Open, Close, High, Low, and Volume**

In [35]:
correlation_data = df[['Open', 'Close', 'High', 'Low', 'Volume']].corr()

In [36]:
correlation_data

,Open,Close,High,Low,Volume
Open,1.000000,0.999668,0.999836,0.999843,-0.214690
Close,0.999668,1.000000,0.999825,0.999841,-0.215027
High,0.999836,0.999825,1.000000,0.999760,-0.212102
Low,0.999843,0.999841,0.999760,1.000000,-0.217666
Volume,-0.214690,-0.215027,-0.212102,-0.217666,1.000000


In [37]:
fig_corr = go.Figure(
    data=go.Heatmap(
        z=correlation_data.values,
        x=correlation_data.columns,
        y=correlation_data.columns,
        colorscale='Viridis',
        colorbar=dict(title="Correlation"),
    ),
    layout=go.Layout(
        title="Correlation Heatmap",
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
    )
)
fig_corr.show()

**Volume vs Price Volatility**

In [38]:
df['Volatility'] = df['High'] - df['Low']

fig_volatility_volume = go.Figure(
    data=go.Scatter(
        x=df['Volume'],
        y=df['Volatility'],
        mode='markers',
        marker=dict(color='cyan', size=5),
        name='Volume vs Volatility',
    ),
    layout=go.Layout(
        title="Volume vs Price Volatility",
        xaxis=dict(title="Volume"),
        yaxis=dict(title="Volatility (High - Low)"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
    )
)
fig_volatility_volume.show()

**Dividends vs Stock Performance (Close Price)**

In [39]:
fig_dividends_performance = go.Figure(
    data=go.Scatter(
        x=df['Dividends'],
        y=df['Close'],
        mode='markers',
        marker=dict(color='lime', size=5),
        name='Dividends vs Close Price',
    ),
    layout=go.Layout(
        title="Dividends vs Stock Performance",
        xaxis=dict(title="Dividends"),
        yaxis=dict(title="Close Price"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
    )
)
fig_dividends_performance.show()

**Stock Splits vs Close Price**

In [40]:
fig_splits_performance = go.Figure(
    data=go.Scatter(
        x=df['Stock Splits'],
        y=df['Close'],
        mode='markers',
        marker=dict(color='magenta', size=5),
        name='Stock Splits vs Close Price',
    ),
    layout=go.Layout(
        title="Stock Splits vs Stock Performance",
        xaxis=dict(title="Stock Splits"),
        yaxis=dict(title="Close Price"),
        plot_bgcolor='black',
        paper_bgcolor='black',
        font=dict(color='white'),
    )
)
fig_splits_performance.show()

### Comparative Analysis

**Compare yearly average prices and volumes**

In [41]:
yearly_data = df.groupby(df['Date'].dt.year).agg(
              avg_price=('Close', 'mean'),
              avg_volume=('Volume', 'mean')).reset_index()

In [42]:
fig_yearly_avg = go.Figure()

# Line for yearly average prices
fig_yearly_avg.add_trace(
    go.Scatter(
        x=yearly_data['Date'],
        y=yearly_data['avg_price'],
        mode='lines+markers',
        line=dict(color='cyan'),
        name='Average Price'
    )
)

# Line for yearly average volumes
fig_yearly_avg.add_trace(
    go.Scatter(
        x=yearly_data['Date'],
        y=yearly_data['avg_volume'],
        mode='lines+markers',
        line=dict(color='magenta'),
        name='Average Volume'
    )
)

fig_yearly_avg.update_layout(
    title="Yearly Average Prices and Volumes",
    xaxis=dict(title="Year"),
    yaxis=dict(title="Average"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
)
fig_yearly_avg.show()

**Price Trends Before and After Significant Events**

In [43]:
significant_years = [2015, 2020]
event_data = df[df['Date'].dt.year.isin(significant_years)].copy()

In [44]:
fig_event_trends = go.Figure()

for year in significant_years:
    yearly_trend = event_data[event_data['Date'].dt.year == year]
    fig_event_trends.add_trace(
        go.Scatter(
            x=yearly_trend['Date'],
            y=yearly_trend['Close'],
            mode='lines+markers',
            name=f"Price Trend - {year}"
        )
    )

fig_event_trends.update_layout(
    title="Price Trends Before and After Significant Events",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Close Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
)
fig_event_trends.show()

**Compare Market Performance During Bull and Bear Periods**

In [45]:
bull_market = df[(df['Date'] >= '2010-01-01') & (df['Date'] <= '2014-12-31')]
bear_market = df[(df['Date'] >= '2015-01-01') & (df['Date'] <= '2019-12-31')]

In [46]:
fig_bull_bear = go.Figure()

# Bull market
fig_bull_bear.add_trace(
    go.Scatter(
        x=bull_market['Date'],
        y=bull_market['Close'],
        mode='lines',
        line=dict(color='lime'),
        name='Bull Market (2010-2014)'
    )
)

# Bear market
fig_bull_bear.add_trace(
    go.Scatter(
        x=bear_market['Date'],
        y=bear_market['Close'],
        mode='lines',
        line=dict(color='red'),
        name='Bear Market (2015-2019)'
    )
)

fig_bull_bear.update_layout(
    title="Market Performance During Bull and Bear Periods",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Close Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
)
fig_bull_bear.show()

### Highs and Lows

In [47]:
# 1. Identify record high and low prices
record_high = df.loc[df['High'].idxmax()]
record_low = df.loc[df['Low'].idxmin()]

# Highlighting periods of prolonged high or low performance
# Define thresholds for prolonged high/low performance
high_threshold = df['High'].mean() + df['High'].std()
low_threshold = df['Low'].mean() - df['Low'].std()

high_periods = df[df['High'] >= high_threshold]
low_periods = df[df['Low'] <= low_threshold]

**Record High and Low Prices**

In [48]:
fig_high_low = go.Figure()

# Line chart of High prices
fig_high_low.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['High'],
        mode='lines',
        line=dict(color='lime'),
        name='High Prices'
    )
)

# Line chart of Low prices
fig_high_low.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Low'],
        mode='lines',
        line=dict(color='red'),
        name='Low Prices'
    )
)

# Add markers for record high and low
fig_high_low.add_trace(
    go.Scatter(
        x=[record_high['Date']],
        y=[record_high['High']],
        mode='markers+text',
        marker=dict(color='cyan', size=10),
        text=['Record High'],
        textposition='top center',
        name='Record High'
    )
)

fig_high_low.add_trace(
    go.Scatter(
        x=[record_low['Date']],
        y=[record_low['Low']],
        mode='markers+text',
        marker=dict(color='magenta', size=10),
        text=['Record Low'],
        textposition='bottom center',
        name='Record Low'
    )
)

fig_high_low.update_layout(
    title="Record High and Low Prices with Dates",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
)
fig_high_low.show()

**Prolonged High and Low Performance**

In [49]:
fig_prolonged = go.Figure()

# High threshold line
fig_prolonged.add_trace(
    go.Scatter(
        x=df['Date'],
        y=[high_threshold] * len(df),
        mode='lines',
        line=dict(color='lime', dash='dash'),
        name='High Threshold'
    )
)

# Low threshold line
fig_prolonged.add_trace(
    go.Scatter(
        x=df['Date'],
        y=[low_threshold] * len(df),
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='Low Threshold'
    )
)

# Highlighted High Periods
fig_prolonged.add_trace(
    go.Scatter(
        x=high_periods['Date'],
        y=high_periods['High'],
        mode='markers',
        marker=dict(color='cyan', size=6),
        name='Prolonged High Performance'
    )
)

# Highlighted Low Periods
fig_prolonged.add_trace(
    go.Scatter(
        x=low_periods['Date'],
        y=low_periods['Low'],
        mode='markers',
        marker=dict(color='magenta', size=6),
        name='Prolonged Low Performance'
    )
)

fig_prolonged.update_layout(
    title="Prolonged High and Low Performance Periods",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
)
fig_prolonged.show()

### Moving Averages

In [50]:
# Calculate Moving Averages
df['50_MA'] = df['Close'].rolling(window=50).mean()
df['100_MA'] = df['Close'].rolling(window=100).mean()
df['200_MA'] = df['Close'].rolling(window=200).mean()

# Identify Golden Crosses and Death Crosses
golden_crosses = df[(df['50_MA'] > df['200_MA']) & (df['50_MA'].shift(1) <= df['200_MA'].shift(1))]
death_crosses = df[(df['50_MA'] < df['200_MA']) & (df['50_MA'].shift(1) >= df['200_MA'].shift(1))]

**Moving Averages and Crosses**

In [51]:
fig_moving_averages = go.Figure()

# Plot Close Prices
fig_moving_averages.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Close'],
        mode='lines',
        line=dict(color='white'),
        name='Close Price'
    )
)

# Plot 50-Day Moving Average
fig_moving_averages.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['50_MA'],
        mode='lines',
        line=dict(color='lime'),
        name='50-Day MA'
    )
)

# Plot 100-Day Moving Average
fig_moving_averages.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['100_MA'],
        mode='lines',
        line=dict(color='cyan'),
        name='100-Day MA'
    )
)

# Plot 200-Day Moving Average
fig_moving_averages.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['200_MA'],
        mode='lines',
        line=dict(color='magenta'),
        name='200-Day MA'
    )
)

# Plot Golden Crosses
fig_moving_averages.add_trace(
    go.Scatter(
        x=golden_crosses['Date'],
        y=golden_crosses['Close'],
        mode='markers+text',
        marker=dict(color='yellow', size=10),
        text=['Golden Cross'] * len(golden_crosses),
        textposition='top center',
        name='Golden Cross'
    )
)

# Plot Death Crosses
fig_moving_averages.add_trace(
    go.Scatter(
        x=death_crosses['Date'],
        y=death_crosses['Close'],
        mode='markers+text',
        marker=dict(color='red', size=10),
        text=['Death Cross'] * len(death_crosses),
        textposition='bottom center',
        name='Death Cross'
    )
)
fig_moving_averages.update_layout(
    title="Moving Averages with Golden and Death Crosses",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
)
fig_moving_averages.show()

### Anomaly Detection

In [52]:
# Calculate percentage changes for Close prices and Volume
df['Price_Change'] = df['Close'].pct_change() * 100  # Percentage change in price
df['Volume_Change'] = df['Volume'].pct_change() * 100  # Percentage change in volume

# Define thresholds for anomalies (3 standard deviations from the mean)
price_threshold = 3 * df['Price_Change'].std()
volume_threshold = 3 * df['Volume_Change'].std()

# Identify anomalies
price_anomalies = df[np.abs(df['Price_Change']) > price_threshold]
volume_anomalies = df[np.abs(df['Volume_Change']) > volume_threshold]

**Anomalies in price changes**

In [53]:
fig_price_anomalies = go.Figure()

# Normal Price Change
fig_price_anomalies.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Price_Change'],
        mode='lines',
        line=dict(color='cyan'),
        name='Price Change (%)'
    )
)

# Anomalies in Price Change
fig_price_anomalies.add_trace(
    go.Scatter(
        x=price_anomalies['Date'],
        y=price_anomalies['Price_Change'],
        mode='markers+text',
        marker=dict(color='red', size=8),
        text=["Anomaly"] * len(price_anomalies),
        textposition='top center',
        name='Price Anomaly'
    )
)

# Update Layout
fig_price_anomalies.update_layout(
    title="Anomaly Detection: Extreme Price Changes",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Price Change (%)"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
)
fig_price_anomalies.show()

**Anomalies in volume changes**

In [54]:
fig_volume_anomalies = go.Figure()

# Normal Volume Change
fig_volume_anomalies.add_trace(
    go.Scatter(
        x=df['Date'],
        y=df['Volume_Change'],
        mode='lines',
        line=dict(color='lime'),
        name='Volume Change (%)'
    )
)

# Anomalies in Volume Change
fig_volume_anomalies.add_trace(
    go.Scatter(
        x=volume_anomalies['Date'],
        y=volume_anomalies['Volume_Change'],
        mode='markers+text',
        marker=dict(color='orange', size=8),
        text=["Anomaly"] * len(volume_anomalies),
        textposition='top center',
        name='Volume Anomaly'
    )
)
fig_volume_anomalies.update_layout(
    title="Anomaly Detection: Extreme Volume Changes",
    xaxis=dict(title="Date"),
    yaxis=dict(title="Volume Change (%)"),
    plot_bgcolor='black',
    paper_bgcolor='black',
    font=dict(color='white'),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="center", x=0.5)
)
fig_volume_anomalies.show()